# Linear Regression in TensorFlow: The California Housing Dataset

In what follows we will use a small dataset to simulate a more realistic scenario. The California Housing dataset contains 20,640 entries and 8 predictors. We first pre-process the dataset by standardizing the predictors, and we save it into an HDF5 file. As a second step, we find the optimal values of the regression parameters using minibatch gradient descent reading from the HDF5 file. In the course of so doing we point out some of the possible pitfalls.

In [68]:
import h5py
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_california_housing
%matplotlib inline
tf.reset_default_graph()

We start extracting the data from the California Housing dataset and standardizing the features. We also add a column of "biases" that do not undergo the normalization step. Following the practices illustrated by A. Geron, we reshape the targets into a one-column array. We use the `np.c_` function to add a column of ones to the matrix. The syntax is a bit surprising, as shown below.

In [69]:
housing = fetch_california_housing()
X_unsc = housing.data # Unscaled data
y_ = housing.target.reshape(-1, 1) # 2D array instead of 1D
ssc = StandardScaler()
X_sc = ssc.fit_transform(X_unsc)
X_sc = np.c_[np.ones(X_sc.shape[0]), X_sc] # Add the bias column

We have created an unscaled version of the data, a scaled version, and a scaled version with an additional column of ones, to represent the intercept or bias factor. All these objects, so far, are flot64.

In [70]:
X_unsc.dtype, X_sc.dtype, y_.dtype

(dtype('float64'), dtype('float64'), dtype('float64'))

We create a couple of constant, float32 objects, and use the normal equations to find the best fit, rather than using SGD. This shows how to create TensorFlow with a specific `dtype`, and exposes some linear algebra functions. We don't need to create a variable storing the value of the $\theta$ parameters, as this will be obtained directly from the matrix operations, and namely from the normal equation: $$\theta = (X^T X)^{-1} X^T y$$

In [71]:
tf.reset_default_graph()
X = tf.constant(X_sc, dtype=tf.float32, name='X') # Store as float 32
y = tf.constant(y_, dtype=tf.float32, name='y')    # Same
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    res = theta.eval()
print(res)

[[ 2.06856298]
 [ 0.82961965]
 [ 0.11875178]
 [-0.26552707]
 [ 0.30569667]
 [-0.00450281]
 [-0.03932635]
 [-0.8998825 ]
 [-0.87053877]]


The results above shows what we should be aiming for when trying to estimate the parameter $\theta$ via gradient descent.

## Fitting the linear model with SGD

In this section we try to estimate the model parameters using Stochastic Gradient Descent. In the previous example $\theta$ was directly calculated via the normal equation. In this case we define it as a variable and initialize it with random values from a uniform distribution. The predicted values are obtained as $\hat y = X^T \theta$ and the error is given by $e = y - \hat y$. Note that `tf.square(error)` below will return an array, and to have a single number we need to summarize it, taking its mean. This is performed by `tf.reduce_mean()`.

In [72]:
tf.reset_default_graph()
m, n = X_sc.shape

X = tf.constant(X_sc, dtype=tf.float32, name='X') # Store as float 32
y = tf.constant(y_, dtype=tf.float32, name='y')    # Same
theta = tf.Variable(tf.random_uniform(shape=[n, 1], minval=-1, maxval=1), name='theta')
y_pred = tf.matmul(X, theta, name='y_pred')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')

To optimize the MSE, we make use of the `GradientDescentOptimizer` function from `tf.train`. We add a `training_op` node to the graph that calls the `optimizer` instance of the gradient descent optimizer.

In [73]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
training_op = optimizer.minimize(mse)
init = tf.global_variables_initializer() # We have a variable to initialize

NUM_EPOCHS = 200

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(NUM_EPOCHS):
        sess.run(training_op)
    res = theta.eval()

print(res)

[[  2.06855774e+00]
 [  8.75750244e-01]
 [  1.31303951e-01]
 [ -3.45384002e-01]
 [  3.68520677e-01]
 [ -5.35451574e-04]
 [ -4.13717031e-02]
 [ -7.65064240e-01]
 [ -7.40735531e-01]]


Note that we need to loop over the number of epochs we decide to train the model, and that within each epoch, we need to run the `training_op`, as this is the node that moves the parameter space towards the minimum. For this approach we have used constants to store `X` and `y`, as we haven't used mini-batch gradient descent. If we want to use it, however, we need to create some temporary storage for each batch, *i.e.*, we need *placeholders*.

In [96]:
tf.reset_default_graph()

BATCH_SIZE = 100
N_BATCHES = int(np.ceil(m / BATCH_SIZE))

X = tf.placeholder(shape=(None, n), dtype=tf.float32, name='X')
y = tf.placeholder(shape=(None, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform(shape=[n, 1], minval=-1, maxval=1), name='theta')
y_pred = tf.matmul(X, theta, name='y_pred')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

We need a function to produce the data batches.

In [97]:
def gen_batch(X, y):
    start = 0
    for batch in range(N_BATCHES):
        start = start + batch * BATCH_SIZE
        end = start + BATCH_SIZE
        X_batch = X[start:stop]
        y_batch = y[start:stop]
        yield X_batch, y_batch

The batch generator will return a batch of the specified size for each epoch. An improvement could be shuffling the samples during the generation.

In [99]:
with tf.Session() as sess:
    sess.run(init) # Re-initialize theta
    for epoch in range(NUM_EPOCHS):
        if epoch % 50 == 0:
            print("Epoch: {}".format(epoch))
        for batch in range(N_BATCHES):
            Xb, yb = next(batch_generator)
            sess.run(training_op, feed_dict={X: Xb, y: yb})
    best_theta = theta.val()

Epoch: 0
Epoch: 50
Epoch: 100
Epoch: 150


NameError: name 'theta_eval' is not defined

## Reading the data from an HDF5 file

We now create the HDF5 dataset that will store the data. Note that the data are of `dtype float64`. We don't convert them into `float32`, as this turns out not to be necessary. However, if our data were of much larger size, it could be a good idea to save a `float32` version to save memory. Note also that once we have created the file handler `hf` we can create datasets using the same name (`dset`, in our case). 

In [88]:
hf = h5py.File('housing.hdf5', 'w')
dset = hf.create_dataset(name='X', data=X)
dset = hf.create_dataset(name='y', data=y)
hf.close()
del X, X_sc, X_unsc, y, ssc, housing

OSError: Unable to create file (Unable to truncate a file which is already open)

## Fitting a linear regression model with minibatch gradient descent

We reload the data, and fix some parameters for the learning phase.

In [ ]:
hf = h5py.File('housing.hdf5', 'r')
X_tr = hf['X']
y_tr = hf['y']
X_tr

We see that `X_tr` is an HDF5 dataset with 20640 rows and 9 columns. We can however already slice it and obtain the values without using the `[...]` selector. Note also that it is enough to specify a *slice*, and not both axis (like in `X_tr[:5, :]`).

In [ ]:
X_tr[:5]

In [ ]:
m, n = X_tr.shape
learning_rate = 0.01
batch_size = 100
num_epochs = 10

We need to create a batch-generating function. Normally the function would return a generator. In this simplified case, we return the batches directly. In Geron's book the batches are created by producing random indices. Our approach here is a bit different, and takes directly the indices of the minibatches, that are computed inside the session. We slice inside the function, and we get, as argument, the start and end position of the slice.

In [ ]:
def get_batch(start, stop):
    return X_tr[slice(start, stop)], y_tr[slice(start, stop)]

We can test whether the function works using a small index

In [ ]:
tmp_x, tmp_y = get_batch(start=0, stop=5)
tmp_x.shape, tmp_y.shape

### Construction of the graph

Our graph is very simple. We have two placeholders for the `X` and the `y` and a variable that stores the values of `theta` (9 components). We initialize the variable with uniform values between -1 and 1. The predicted values are simply obtained by multiplying `X` by `theta`. We use a Gradient Descent Optimizer and we use it to minimize the Mean Squared Error. 

In [ ]:
graph = tf.Graph()
with graph.as_default():
    X = tf.placeholder(dtype=tf.float32, shape=(None, n), name='X')
    y = tf.placeholder(dtype=tf.float32, shape=(None, 1), name='y')
    theta = tf.Variable(tf.random_uniform(
        [n, 1], -1.0, 1.0, seed=42), name='theta')
    y_pred = tf.matmul(X, theta, name='y_pred')
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name='mse')
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(mse)
    init = tf.global_variables_initializer()

### Execution phase

For the execution phase we compute the number of batches from the `batch_size` and the total number of observations. We generate a start and end position for each minibatch and generate the minibatches with the `get_batch` function.

In [ ]:
with tf.Session(graph=graph) as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        print("Epoch: {}".format(epoch))
        num_batches = m // batch_size
        for batch_index in range(num_batches):
            start = batch_size * batch_index
            stop = batch_size * (batch_index + 1)
            X_batch, y_batch = get_batch(start, stop)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
    best_theta = theta.eval()
print(best_theta)
hf.close()